In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, model_selection, pipeline, manifold, preprocessing,feature_selection 
from colorama import Fore, Back, Style
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from colorama import Fore, Back, Style
from sklearn.feature_extraction.text import TfidfTransformer 

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/semihyazici/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/semihyazici/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
PATH = "/home/semihyazici/Desktop/Semih/Code_Resources/NLP-Proje/train.csv"

In [3]:
data = pd.read_csv(PATH,)


In [4]:
data['toxic'].unique()

array([0, 1])

In [5]:
neutral = data[(data["toxic"] != 1) & (data["severe_toxic"] != 1)& (data["obscene"] != 1)
    & (data["threat"] != 1)& (data["insult"] != 1)& (data["identity_hate"] != 1)]
neutral['class'] = 1

/tmp/ipykernel_101069/2843459197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neutral['class'] = 1


In [6]:
negative = data[(data["toxic"] != 0) | (data["severe_toxic"] != 0)| (data["obscene"] != 0)
    | (data["threat"] != 0)| (data["insult"] != 0)| (data["identity_hate"] != 0)]
negative['class'] = 0

/tmp/ipykernel_101069/4147647893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative['class'] = 0


# Undersampling

In [7]:
neutral.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'class'],
      dtype='object')

In [11]:
sample_num = negative.shape[0]
neutral = neutral.sample(sample_num)



# Preprocess

In [12]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
    text = " ".join(lst_text)
    return text

In [13]:
def textCleaner(df = None , src = 'comment_text' ,dst = 'text_clean',stop_words = 'english'):
    
    if df is None:
        raise TypeError("Data Frame cannot be type 'None'")
    
    try:
        lst_stopwords = nltk.corpus.stopwords.words(stop_words)
    except:
        raise Exception( "'" + stop_words +"'"+" is not a valid type.")
    df[dst] = df[src].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))

In [14]:
neutral.reset_index(drop = True, inplace = True)
textCleaner(df = neutral,stop_words = 'english',src='comment_text')
textCleaner(df = negative,stop_words = 'english',src='comment_text')

col2remove =  ['id','toxic','severe_toxic',
               'obscene','threat','insult','identity_hate','comment_text']
neutral.drop(col2remove, inplace = True, axis = 1) 
negative.drop(col2remove, inplace = True, axis = 1) 
negative.reset_index(drop = True, inplace = True)

/tmp/ipykernel_101069/2085000868.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[dst] = df[src].apply(lambda x:
/home/semihyazici/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
X_data, y_data = np.array(negative['text_clean'].append(neutral['text_clean'])) , np.array(negative['class'].append(neutral['class']))
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000,ngram_range=(1,1))


In [16]:
X_train,X_val,y_train,y_val = train_test_split(X_data,y_data,test_size=0.1,shuffle=True,)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val,test_size=0.5,shuffle=True,)

In [17]:
corpus = X_train
vectorizer.fit(corpus)
x_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_
X_names = vectorizer.get_feature_names()
p_value_limit = 0.95
dtf_features = pd.DataFrame()
for cat in np.unique(y_train):
    chi2, p = feature_selection.chi2(x_train, y_train==cat)
    dtf_features = dtf_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    dtf_features = dtf_features.sort_values(["y","score"], 
                    ascending=[True,False])
    dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
X_names = dtf_features["feature"].unique().tolist()
vectorizer = feature_extraction.text.TfidfVectorizer(vocabulary=X_names)
vectorizer.fit(corpus)
X_tr = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

/home/semihyazici/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Naive Bayes Approach

In [110]:
model = MultinomialNB()
model.fit(X_tr, (y_train))

MultinomialNB()

In [99]:
#Train başarısı
sum(model.predict_proba(X_tr).argmax(axis=1) == y_train) / len(y_train)

0.8887861667522684

In [100]:
#Validation Başarısı
sum(model.predict_proba(vectorizer.transform(X_val)).argmax(axis=1) == y_val) / len(y_val)

0.876078914919852

In [101]:
#Test başarısı
sum(model.predict_proba(vectorizer.transform(X_test)).argmax(axis=1) == y_test) / len(y_test)

0.8798521256931608

In [102]:
#Karışıklık matrisi
confusion_matrix(model.predict_proba(vectorizer.transform(X_test)).argmax(axis=1),y_test)

array([[721, 108],
       [ 87, 707]])

In [25]:
def find_prob_words(txt,vectorizer,model):
    tokens = vectorizer.transform(txt)
    token_nums = vectorizer.inverse_transform(tokens)[0]
    ltl_map = model.feature_log_prob_
    vocab = vectorizer.vocabulary
    word_map = {token_nums[i]:ltl_map[:,token].argmax()   for i,token in enumerate(tokens.indices)}
    return [word  for word,val in word_map.items() if val == 0]

In [31]:
class WordHighlight():
    
    def __init__(self,highlight = True):
        
        self.highlight = highlight
        
    def __call__(self,original_txt,res):    
        color = Back.RED
        out_sent = ''
        
        if type(original_txt) != str:
            original_txt = str(original_txt).split()
        
        
        for word in original_txt:

            checker = sum([1 if resp_word in word else 0 for resp_word in res])
            if checker:
                if self.highlight:
                    out_sent += color+  word + Style.RESET_ALL+' '
                else:
                    out_sent += '*** '
                    
            else:
                out_sent += word + ' '

        return out_sent

In [111]:
sentence = np.array(["i dont know who that bastard was but he will pay with blood i'll kill him"])
df = pd.DataFrame({"class":"0","comment_text":sentence})
textCleaner(df = df,stop_words = 'english',src='comment_text')
tokens = vectorizer.transform(df.text_clean)

res = find_prob_words(df.text_clean,vectorizer=vectorizer,model=model)

a = WordHighlight(highlight=True)
print(a(sentence[0],res))

i dont know who that bastard was but he will pay with blood i'll kill him 


# With RNN/LSTM/GRU

In [18]:
import torch 
import torch.nn as nn
from transformers import DistilBertTokenizer
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
pretrained = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(pretrained)
maxlen = 150
limit = 30

In [20]:

indexes = []
for i,text in enumerate(X_val):
    if len(text.split()) > limit:
        indexes.append(i)
X_val = np.delete(X_val,indexes)
y_val = np.delete(y_val,indexes)

In [21]:
indexes = []
for i,text in enumerate(X_test):
    if len(text.split()) > limit:
        indexes.append(i)
X_test = np.delete(X_test,indexes)
y_test = np.delete(y_test,indexes)

In [22]:
indexes = []
for i,text in enumerate(corpus):
    if len(text.split()) > limit:
        indexes.append(i)
corpus = np.delete(corpus,indexes)
y_train = np.delete(y_train,indexes)

In [23]:
class Dataset():
    
    def __init__(self,tokenizer,sentences,labels,maxlen,device ):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.labels = labels
        self.maxlen = maxlen
    def __getitem__(self,index):
        
        return self.tokenizer(self.sentences[index],max_length=self.maxlen,padding='max_length',
                              return_tensors="pt").to(device),torch.tensor(self.labels[index],
                                                                                        device = device)
    def __len__(self,):
        return len(self.sentences)
        

In [24]:
class RNNModel(nn.Module):
    
    def __init__(self,n_vocab,d_model,maxlen,bidirectional = True):
        
        super().__init__()
        self.d_model = d_model
        self.times = (2 if bidirectional  else 1)
        
        
        
        self.embedding = nn.Embedding(n_vocab,self.d_model,)
        self.rnn = nn.RNN(self.d_model,self.d_model,1,batch_first = True,
                          bidirectional = bidirectional)
        self.attention =  nn.Linear(self.d_model * self.times,1)
        self.softmax = nn.Softmax(dim=1)
        self.classifier = nn.Sequential(
            nn.Linear(self.times * d_model,256),
            nn.LayerNorm(256),
            nn.Dropout(.15),
            nn.ReLU()    ,
            nn.Linear(256,128),
            nn.LayerNorm(128),
            nn.Dropout(.15),
            nn.ReLU(),
            nn.Linear(128,2))
    
    
    def _generate_initial_hidden(self,batch_size):
        return torch.zeros((self.times,batch_size,self.d_model ),device = device,
                           dtype=torch.float32)

    def forward(self,x,mask):
        embeds = self.embedding(x)
        batch_size = embeds.shape[0]
        initial_hidden = self._generate_initial_hidden(batch_size)
        h,c = self.rnn(embeds,initial_hidden)
        mask = mask.permute(0,2,1)
        attentions = self.softmax(self.attention(h * mask))
        h = attentions * h
        outs = self.classifier(h[:,-1,:])
        #outs = self.classifier(torch.rand((batch_size,h.shape[-1])))
        return outs,attentions

In [23]:
class GRUModel(nn.Module):
    
    def __init__(self,n_vocab,d_model,maxlen,bidirectional = True):
        
        super().__init__()
        self.d_model = d_model
        self.times = (2 if bidirectional  else 1)
        
        
        
        self.embedding = nn.Embedding(n_vocab,self.d_model,)
        self.gru = nn.GRU(self.d_model,self.d_model,1,batch_first = True,
                          bidirectional = bidirectional)
        self.attention =  nn.Linear(self.d_model * self.times,1)
        self.softmax = nn.Softmax(dim=1)
        self.classifier = nn.Sequential(
            nn.Linear(self.times * d_model,256),
            nn.LayerNorm(256),
            nn.Dropout(.15),
            nn.ReLU()    ,
            nn.Linear(256,128),
            nn.LayerNorm(128),
            nn.Dropout(.15),
            nn.ReLU(),
            nn.Linear(128,2))
    
    
    def _generate_initial_hidden(self,batch_size):
        return torch.zeros((self.times,batch_size,self.d_model ),device = device,
                           dtype=torch.float32)

    def forward(self,x,mask):
        embeds = self.embedding(x)
        batch_size = embeds.shape[0]
        initial_hidden = self._generate_initial_hidden(batch_size)
        h,c = self.gru(embeds,initial_hidden)
        mask = mask.permute(0,2,1)
        attentions = self.softmax(self.attention(h * mask))
        h = attentions * h
        outs = self.classifier(h[:,-1,:])
        #outs = self.classifier(torch.rand((batch_size,h.shape[-1])))
        return outs,attentions

In [24]:
class LSTMModel(nn.Module):
    
    def __init__(self,n_vocab,d_model,maxlen,bidirectional = True):
        
        super().__init__()
        self.d_model = d_model
        self.times = (2 if bidirectional  else 1)
        
        
        
        self.embedding = nn.Embedding(n_vocab,self.d_model,)
        self.lstm = nn.LSTM(self.d_model,self.d_model,1,batch_first = True,
                          bidirectional = bidirectional)
        self.attention =  nn.Linear(self.d_model * self.times,1)
        self.softmax = nn.Softmax(dim=1)
        self.classifier = nn.Sequential(
            nn.Linear(self.times * d_model,256),
            nn.LayerNorm(256),
            nn.Dropout(.15),
            nn.ReLU()    ,
            nn.Linear(256,128),
            nn.LayerNorm(128),
            nn.Dropout(.15),
            nn.ReLU(),
            nn.Linear(128,2))
    
    
    def _generate_initial_hidden(self,batch_size):
        return (torch.zeros((self.times,batch_size,self.d_model ),device = device,
                           dtype=torch.float32),
                torch.zeros((self.times,batch_size,self.d_model ),device = device,
                           dtype=torch.float32))

    def forward(self,x,mask):
        embeds = self.embedding(x)
        batch_size = embeds.shape[0]
        h0,c0 = self._generate_initial_hidden(batch_size)
        h,c = self.lstm(embeds,(h0,c0))
        mask = mask.permute(0,2,1)
        attentions = self.softmax(self.attention(h * mask))
        h = attentions * h
        outs = self.classifier(h[:,-1,:])
        #outs = self.classifier(torch.rand((batch_size,h.shape[-1])))
        return outs,attentions

In [25]:
model = RNNModel(n_vocab = tokenizer.vocab_size,
              d_model= 256,
              maxlen = maxlen,
              bidirectional = True).to(device)
model.load_state_dict(torch.load("rnn_params.pth"))

<All keys matched successfully>

In [78]:
model = GRUModel(n_vocab = tokenizer.vocab_size,
              d_model= 256,
              maxlen = maxlen,
              bidirectional = False).to(device)

In [83]:
model = LSTMModel(n_vocab = tokenizer.vocab_size,
              d_model= 256,
              maxlen = maxlen,
              bidirectional = False).to(device)

In [26]:
lr = 0.001
batch_size = 32
n_epochs = 5
n_iter = 1000
print_per_iter = 20#n_iter // 20

In [28]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr = lr,)
data = Dataset(tokenizer,corpus,y_train,maxlen,device)
dataloader = torch.utils.data.DataLoader(dataset=data,batch_size=batch_size,shuffle=True) 

data_val = Dataset(tokenizer,X_val,y_val,maxlen,device)
dataloader_val = torch.utils.data.DataLoader(dataset=data_val,batch_size=batch_size,shuffle=True) 
data_test = Dataset(tokenizer,X_test,y_test,maxlen,device)
dataloader_test = torch.utils.data.DataLoader(dataset=data_test,batch_size=batch_size,shuffle=True) 
model.train()

RNNModel(
  (embedding): Embedding(30522, 256)
  (rnn): RNN(256, 256, batch_first=True, bidirectional=True)
  (attention): Linear(in_features=512, out_features=1, bias=True)
  (softmax): Softmax(dim=1)
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): Dropout(p=0.15, inplace=False)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (6): Dropout(p=0.15, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [29]:
for epoch in (range(n_epochs)):
    model.train()
    
    cum_loss = 0.0
    train_acc = 0
    for iteration in tqdm(range(n_iter)):
        try:

            batch,labels =  next(iter(dataloader))
            sentences,masks = batch['input_ids'],batch['attention_mask']
        except KeyboardInterrupt:
            raise KeyboardInterrupt()
        except Exception as e:
            pass
        optim.zero_grad()

        preds,_ = model(sentences.squeeze(1),masks)
        
        loss = loss_fn(preds,labels)
        train_acc += torch.sum(preds.argmax(dim=1) == labels)
        cum_loss += loss.item()
        
        loss.backward()
        optim.step()

    
    
    final_train_acc = train_acc/(n_iter * batch_size)
    
    test_acc = 0
    model.eval()

    for batch,labels in dataloader_val:
        sentences,masks = batch['input_ids'],batch['attention_mask']
        with torch.no_grad():
            preds,_ = model(sentences.squeeze(1),masks)
        test_acc += torch.sum(preds.argmax(dim=1) == labels)
    test_acc = test_acc/len(data_test)
    
    print(f"Epoch = {epoch} Loss = {cum_loss} Train Accuracy = {final_train_acc} Val Accuracy = {test_acc}")
    

100%|███████████████████████████████████████| 1000/1000 [00:45<00:00, 21.79it/s]


Epoch = 0 Loss = 357.7144889831543 Train Accuracy = 0.8345937728881836 Val Accuracy = 0.8259783387184143


100%|███████████████████████████████████████| 1000/1000 [00:44<00:00, 22.30it/s]


Epoch = 1 Loss = 189.9455473460257 Train Accuracy = 0.9329375624656677 Val Accuracy = 0.8459616899490356


100%|███████████████████████████████████████| 1000/1000 [00:52<00:00, 19.14it/s]


Epoch = 2 Loss = 125.1020605545491 Train Accuracy = 0.9587187767028809 Val Accuracy = 0.8376352787017822


  2%|▉                                        | 24/1000 [00:02<01:29, 10.86it/s]


KeyboardInterrupt: 

In [30]:
test_acc = 0
preds_mem = []
true_mem = []
model.eval()
    
for batch,labels in dataloader_test:
    sentences,masks = batch['input_ids'],batch['attention_mask']
    with torch.no_grad():
        preds,_ = model(sentences.squeeze(1),masks)
        preds_mem.append(preds.argmax(dim=1))
    test_acc += torch.sum(preds.argmax(dim=1) == labels)
    true_mem.append(labels)
test_acc = test_acc/len(data_test)
print(test_acc)

tensor(0.8843, device='cuda:0')


In [28]:
def attention2word(tokenizer,tokens,attention,top = 5):
    items = attention[0,:,0].topk(top).indices.detach().cpu()
    resp_words = [tokenizer.ids_to_tokens[tokens[0][item].detach().cpu().item()] for item in items]
    return resp_words
    
    

In [44]:
def test(text):
    sentence = np.array([text])

    df = pd.DataFrame({"class":"0","comment_text":sentence})
    textCleaner(df = df,stop_words = 'english',src='comment_text')

    sent = df.text_clean[0]

    batch_test = tokenizer(sent,max_length=maxlen,padding='max_length',
                                  return_tensors="pt").to(device)

    tokens,mask = batch_test['input_ids'],batch_test['attention_mask'].unsqueeze(0)
    model.eval()

    with torch.no_grad():
        pred,att = model(tokens,mask)
    if pred.detach().cpu().argmax(dim=1) == 1:
        print(text)
        return
    tokens = tokens.detach().cpu()
    resp_words = attention2word(tokenizer,tokens,att,top = 3)
    a = WordHighlight(highlight=True)
    print(a(sentence[0],resp_words))

In [45]:
test("i dont know who that bastard was but he will pay with blood i'll kill him")

i dont know who that bastard was but he will pay with blood i'll kill him 


In [46]:
test("i think every life has a value ")

i think every life has a value 


In [49]:
test("That litte bastard is trying to rob the bank")

That litte bastard is trying to rob the bank 


In [50]:
test("That litte pimp is trying to rob the bank")

That litte pimp is trying to rob the bank


In [91]:
confusion_matrix(torch.concat(preds_mem,dim=0).detach().cpu(),torch.concat(true_mem,dim=0).detach().cpu())

array([[590,  73],
       [ 57, 465]])